In [1]:
# Some functions are from or based on work done by Kevin Liao in the below notebook
# https://github.com/KevinLiao159/MyDataSciencePortfolio/blob/master/movie_recommender/movie_recommendation_using_ALS.ipynb

# Intialization
import os
import time

import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

# spark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction, explode, desc
from pyspark.sql.types import StringType, ArrayType
from pyspark.ml.recommendation import ALS
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor, GBTRegressionModel

# data science imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
data_path = 'hdfs:///user/andrew/'

In [3]:
%%time
# Read in data through spark since the data is sored in hadoop and format the columns
# Convert to pandas dataframes for easier and faster manipulation
from pyspark.sql.types import *
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import *
sqlContext = SQLContext(sc)

movies = sqlContext.read.parquet(data_path + 'movie_20m_metadata_OHE_subset')
movies_df = movies.toPandas()
movies_df = movies_df.set_index(movies_df.item_id) # set index so no sorting errors occur

# movies_gp = sqlContext.read.parquet('hdfs:///user/andrew/movie_genre_and_people_metadata_ohe_subset')
movies_gp = movies.drop('title', 'imdb_id', 'imdb_rating', 'imdb_votes', 'metascore', 'runtime', 'year')

ratings = sqlContext.read.parquet(data_path + 'ratings_20m')
ratings = ratings.drop('timestamp')
ratings = ratings.withColumn("userId", ratings["userId"].cast("int"))
ratings = ratings.withColumn("rating", ratings["rating"] * 2) #Multiply by 2 so that values are whole numbers -> values 1 to 10
ratings = ratings.select('userId', 'movieId', 'rating').toDF('user_id', 'item_id', 'label')

CPU times: user 2.65 s, sys: 147 ms, total: 2.8 s
Wall time: 10.4 s


In [4]:
# User input function - takes user input data, strpis it down, and calls other functions on that data
# Takes in user age, gender, occupation (of 20 options - may drop this), list of favorite movies
# All movies in the list of favorite movies will be rated 5 stars
def new_user_input(fav_movies, all_ratings, movies, spark_context, 
                   sqlContext = None, num_recs = 10, movies_gp = None, 
                   movies_df = None):
    # age should be an integer in 1 - 100
    # gender should be M or F
    # fav_movies should be in the form of ["Iron Man", "The Shawshank Redemption", "Robin Hood"]
    #    If there are multiple versions of the movie and the user wishes for one other than the most recent one, they
    #    should specify with a year in parenthesis, like "Robin Hood (1993)"
       
    # Occupation...
    
    # collect favorite movie ids
    print 'Collecting favorite movie IDs'
    movieIds = get_movieId(movies_df, fav_movies)
    print 'Favorite movies in the available set'
    print movies_df[['item_id', 'title', 'year']].loc[movieIds]
    
    print 'Adding ratings to full set'
    # add new user movie ratings to all ratings dataframe
    # all_ratings_updated, user_ratings, user_ratings_binary = add_new_user_to_data(all_ratings, movieIds, spark_context)
    all_ratings_updated, new_user_ratings = add_new_user_to_data(all_ratings, movieIds, spark_context)
    del all_ratings
    
    print 'Creating prediction set'
    # get all unrated movies for user (unnecessary in Spark 2.2+, instead use the recommendForAllUsers(num_to_rec) method)
    all_user_unrated = get_inference_data(all_ratings_updated, movieIds)
    
    print 'Training ALS model'
    # train ALS model, then predict movie ratings
    als = ALS(seed = 42, regParam = 0.1, maxIter = 15, rank = 12, # coldStartStrategy = 'drop', # drops userIds/movieIds from the validation set or test set so that NaNs are not returned
          userCol = "user_id", itemCol = "item_id", ratingCol = "label")
    als_model = als.fit(all_ratings_updated)
    del all_ratings_updated
    
    print 'Making Predictions'
    # keep top 30 predictions
    full_predictions_sorted = als_model.transform(all_user_unrated).sort(desc('prediction'))
    als_top_n_predictions = full_predictions_sorted.take(num_recs)
    # extract movie ids
    als_top_n_ids = [r[1] for r in als_top_n_predictions]
    
    als_movie_recs = movies.filter(movies.item_id.isin(als_top_n_ids)).select('title', 'year')
    print ''
    print 'ALS Recommendations'
    print als_movie_recs.toPandas()
    
    
    # format data for prediction using GBTs
    # create user_id x item_id matrix need to get data in the form of user_id, item_id, label, then pivot
    # filter movies_gp dataframe by the movieIds. pivot new_user_ratings into a vector, 
    # then multiply by the filtered movies_gp dataframe; divide by binarized user ratings; 
    # this should now be a vector of user preferences. 
    # join a OHE age, gender, and possibly occupation, to the user preferences
    user_summary = get_user_preferences(user_ratings = new_user_ratings, movieIds = movieIds, 
                                        movies_gp = movies_gp, sqlContext = sqlContext)
    
    # Extract movie ids from the top 5*num_recs for Gradient Boosted Trees prediction
    als_top_3xn_predictions = full_predictions_sorted.take(3*num_recs)
    als_top_3xn_ids = [r[1] for r in als_top_3xn_predictions]
    all_user_unrated_top_3xn = all_user_unrated.filter(all_user_unrated.item_id.isin(als_top_3xn_ids))
    top_3xn_movies_metadata = movies.filter(movies.item_id.isin(als_top_3xn_ids))
        
    # lastly, replicate the user pref rows for each rated movieId, then join with the filtered movies dataframe
    # (MAKE SURE ALL COLUMNS ARE ORDERED AND NAMED CORRECTLY)
    unrated_with_movie_metadata = all_user_unrated_top_3xn \
                                    .join(top_3xn_movies_metadata, on = 'item_id', how = 'left')
    unrated_with_full_metadata = unrated_with_movie_metadata \
                                    .join(user_summary, on = 'user_id', how = 'left') \
                                    .drop('user_id', 'title', 'imdb_id')
    
    # the GBT model takes in the rows as vectors, so the columns must be converted to the feature space
    unrated_with_full_metadata_rdd = unrated_with_full_metadata.rdd.map(lambda x: (x[0], Vectors.dense(x[1:])))
    unrated_metadata_features = sqlContext.createDataFrame(unrated_with_full_metadata_rdd, schema = ['item_id', 'features'])
    
    # import the GBT model, in this case a GBTRegressionModel with tree depth of 10
    GBTRegD10Model = GBTRegressionModel.load(data_path + 'GBTRegD10Model_20m')
    # use pre-trained GBT model to predict movie ratings
    gbtr_preds = GBTRegD10Model.transform(unrated_metadata_features)
    
    # sort by predicted rating, and keep top recommend top n
    gbtr_top_n_predictions = gbtr_preds.sort(desc('prediction')).take(num_recs)
    # extract movie ids
    gbtr_top_n_ids = [r[0] for r in gbtr_top_n_predictions]

    gbtr_movie_recs = movies.filter(movies.item_id.isin(gbtr_top_n_ids)).select('title', 'year')
    print ''
    print 'GBTR Recommendations'
    print gbtr_movie_recs.toPandas()

In [5]:
def get_movieId(movies_df, fav_movie_list):
    """
    return all movieId(s) of user's favorite movies
    
    Parameters
    ----------
    df_movies: spark Dataframe, movies data
    
    fav_movie_list: list, user's list of favorite movies
    
    Return
    ------
    movieId_list: list of movieId(s)
    """
    movieId_list = []
    for movie in fav_movie_list:
        if movie[0:4] == 'The ':
            movie = movie[4:]
        elif movie[0:3] == 'An ':
            movie = movie[3:]
        elif movie[0:3] == 'La ':
            movie = movie[3:]
        elif movie[0:2] == 'A ':
            movie = movie[3:]

        if movie[-6:-5] == '(':
            year = int(movie[-5:-1])
            movie = movie[0:-7]
            movieIds = movies_df.item_id[(movies_df.title.str.contains(movie)) & (movies_df.year == year)]
            movieId_list.extend(movieIds)
        elif len(movie.split(' ')) == 1:
            movieIds = movies_df.item_id[movies_df.title == movie]
            movieId_list.extend(movieIds)
        else:
            movieIds = movies_df.item_id[movies_df.title.str.contains(movie)]
            movieId_list.extend(movieIds)
    return movieId_list

In [6]:
def add_new_user_to_data(train_data, movieIds, spark_context):
    """
    add new rows with new user, user's movie and ratings to
    existing train data

    Parameters
    ----------
    train_data: Spark DataFrame, ratings data
    
    movieIds: spark DataFrame, single column of movieId(s)

    spark_context: Spark Context object
    
    Return
    ------
    new train data with the new user's rows
    """
    # get new user id
    new_id = train_data.agg({"user_id": "max"}).collect()[0][0] + 1
    # get max rating
    max_rating = train_data.agg({"label": "max"}).collect()[0][0]
    # create new user sdf for max rating
    user_rows_max = [(new_id, movieId, max_rating) for movieId in movieIds]
    new_sdf_max = spark_context.parallelize(user_rows_max).toDF(['user_id', 'item_id', 'label'])
    # return new train data
    return train_data.union(new_sdf_max), new_sdf_max # , new_sdf_binary

In [7]:
def get_inference_data(train_data, movieIds):
    """
    return a rdd with the userid and all movies (except ones in movieId_list)

    Parameters
    ----------
    train_data: spark RDD, ratings data

    df_movies: spark Dataframe, movies data
    
    movieId_list: list, list of movieId(s)

    Return
    ------
    inference data: Spark RDD
    """
    # get new user id
    new_id = train_data.agg({"user_id": "max"}).collect()[0][0]
    
    distinct_unrated_items = ratings.select('item_id').distinct().filter(~col('item_id').isin(movieIds))

    user_unrated = distinct_unrated_items.withColumn('user_id', lit(new_id)).select('user_id', 'item_id')
    return user_unrated

In [8]:
def get_user_preferences(user_ratings, movieIds, movies_gp, sqlContext):        
    #new_user_ratings
    # pivoted_user_ratings = user_ratings.groupBy('user_id').pivot('item_id').agg(avg('label'))
    # pivoted_new_user_ratings_binary = user_ratings_binary.groupBy('user_id').pivot('item_id').agg(avg('label')).drop('user_id')
    pivoted_user_ratings_df = user_ratings.toPandas() \
                                            .pivot(index='user_id', 
                                                   columns='item_id',
                                                   values='label') \
                                            .fillna(0)
    pivoted_user_ratings_df_binary = pivoted_user_ratings_df / pivoted_user_ratings_df
    
    movies_gp_filtered = movies_gp.filter(col('item_id').isin(movieIds))
    movies_gp_filtered_df = movies_gp_filtered.toPandas()
    # movies_gp_filtered_df.item_id = movies_gp_filtered_df.item_id.astype(str) only necessary when pivot was done on spark df
    movies_gp_filtered_df = movies_gp_filtered_df.set_index('item_id')
    
    user_summary_total = pivoted_user_ratings_df.dot(movies_gp_filtered_df)
    user_summary_count = pivoted_user_ratings_df_binary.dot(movies_gp_filtered_df)
    user_summary_avg = (user_summary_total / user_summary_count).fillna(0)
    user_summary_avg = user_summary_avg.add_suffix('_avg_rating').reset_index()
    
    sorted_columns = list(user_summary_avg.columns.sort_values())
    user_summary_sdf = sqlContext.createDataFrame(user_summary_avg[sorted_columns])
    return user_summary_sdf

### Step by Step Walkthrough of Main Function (to show runtime)

In [9]:
%%time
fav_movies = ['Tinker Tailor Soldier Spy', 'Shawshank Redemption', 'Lord of the Rings']
# collect favorite movie ids
print 'Collecting favorite movie IDs'
movieIds = get_movieId(movies_df, fav_movies)
if movies_df is not None:
    print 'Favorite movies in the available set'
    print movies_df[['item_id', 'title', 'year']].loc[movieIds]

Favorite movies in the available set
         item_id                                              title  year
item_id                                                                  
89753      89753                   Tinker Tailor Soldier Spy (2011)  2011
318          318                   Shawshank Redemption, The (1994)  1994
2116        2116                      Lord of the Rings, The (1978)  1978
7153        7153  Lord of the Rings: The Return of the King, The...  2003
4993        4993  Lord of the Rings: The Fellowship of the Ring,...  2001
5952        5952      Lord of the Rings: The Two Towers, The (2002)  2002
CPU times: user 42.6 ms, sys: 15.3 ms, total: 58 ms
Wall time: 54.3 ms


In [10]:
%%time
print 'Adding ratings to full set'
# add new user movie ratings to all ratings dataframe
# all_ratings_updated, user_ratings, user_ratings_binary = add_new_user_to_data(all_ratings, movieIds, spark_context)
all_ratings_updated, user_ratings = add_new_user_to_data(ratings, movieIds, sc)

Adding ratings to full set
CPU times: user 23.2 ms, sys: 7.6 ms, total: 30.8 ms
Wall time: 9.37 s


In [11]:
%%time
print 'Creating prediction set'
# get all unrated movies for user (unnecessary in Spark 2.2+, instead use the recommendForAllUsers(num_to_rec) method)
all_user_unrated = get_inference_data(all_ratings_updated, movieIds)

Creating prediction set
CPU times: user 9.18 ms, sys: 7.63 ms, total: 16.8 ms
Wall time: 3.95 s


In [12]:
%%time
print 'Training ALS model'
# train ALS model, then predict movie ratings
als = ALS(seed = 42, regParam = 0.1, maxIter = 15, rank = 12,
      userCol = "user_id", itemCol = "item_id", ratingCol = "label")
als_model = als.fit(all_ratings_updated)
del all_ratings_updated

Training ALS model
CPU times: user 36.2 ms, sys: 7.64 ms, total: 43.8 ms
Wall time: 21.9 s


In [13]:
%%time
print 'Making Predictions'
# keep top 15 predictions
num_recs = 15
full_predictions = als_model.transform(all_user_unrated)
als_top_n_predictions = full_predictions.sort(desc('prediction')).take(num_recs)
# extract movie ids
als_top_n_ids = [r[1] for r in als_top_n_predictions]

als_movie_recs = movies.filter(movies.item_id.isin(als_top_n_ids)).select('title', 'year')
print 'ALS Recommendations'
print als_movie_recs.toPandas()

Making Predictions
ALS Recommendations
                                                title  year
0                         Unbeatable (Ji zhan) (2013)  2013
1                      Amanece, que no es poco (1989)  1989
2                 National Theatre Live: Frankenstein  2014
3   I Don't Want to Sleep Alone (Hei yan quan) (2006)  2006
4                   Drained (O cheiro do Ralo) (2006)  2006
5   Pit, the Pendulum and Hope, The (Kyvadlo, jAma...  1984
6                Secret of Santa Vittoria, The (1969)  1969
7                                      PlAcido (1961)  1961
8                               Otakus in Love (2004)  2004
9   Welcome Mr. Marshall (Bienvenido Mister Marsha...  1953
10                       Policeman (Ha-shoter) (2011)  2011
11  Marathon Family, The (Maratonci Trce Pocasni K...  1982
12                             Moth, The (Cma) (1980)  1980
13                Front Line, The (Go-ji-jeon) (2011)  2011
14                         Code Name Coq Rouge (1989)  1989
C

In [14]:
%%time
# import GBT model input data format
# Create user_id x item_id matrix need to get data in the form of user_id, item_id, label, then pivot
# filter movies_gp dataframe by the movieIds. pivot new_user_ratings into a vector, 
# then multiply by the filtered movies_gp dataframe; divide by binarized user ratings; 
# this should now be a vector of user preferences. 
# join a OHE age, gender, and possibly occupation, to the user preferences
user_summary_sdf = get_user_preferences(user_ratings = user_ratings, movieIds = movieIds, 
                                        movies_gp = movies_gp, sqlContext = sqlContext)

CPU times: user 104 ms, sys: 35.1 ms, total: 139 ms
Wall time: 4.39 s


In [15]:
%%time
als_top_3xn_predictions = full_predictions.sort(desc('prediction')).take(3*num_recs)
als_top_3xn_ids = [r[1] for r in als_top_3xn_predictions]
all_user_unrated_top_3xn = all_user_unrated.filter(all_user_unrated.item_id.isin(als_top_3xn_ids)) #looks good
top_3xn_movies_metadata = movies.filter(movies.item_id.isin(als_top_3xn_ids)) #looks good

CPU times: user 33.8 ms, sys: 6.34 ms, total: 40.1 ms
Wall time: 4.22 s


In [16]:
%%time
# lastly, replicate the user pref rows for each rated movieId, then join with the filtered movies dataframe
# (MAKE SURE ALL COLUMNS ARE ORDERED AND NAMED CORRECTLY)
unrated_with_movie_metadata = all_user_unrated_top_3xn \
                                .join(top_3xn_movies_metadata, on = 'item_id', how = 'left') #looks good
unrated_with_full_metadata = unrated_with_movie_metadata \
                                .join(user_summary_sdf, on = 'user_id', how = 'left') \
                                .drop('user_id', 'title', 'imdb_id') #looks good

CPU times: user 3.9 ms, sys: 0 ns, total: 3.9 ms
Wall time: 232 ms


In [17]:
%%time
# convert predictors to "features" and it is ready for prediction. 
# features_cols = list(unrated_with_full_metadata.columns)
# features_cols.remove('item_id')

# vecAssembler = VectorAssembler(inputCols = features_cols, outputCol="features")
# unrated_metadata_features = vecAssembler.transform(unrated_with_full_metadata)

unrated_with_full_metadata_rdd = unrated_with_full_metadata.rdd.map(lambda x: (x[0], Vectors.dense(x[1:])))
unrated_metadata_features = sqlContext.createDataFrame(unrated_with_full_metadata_rdd, schema = ['item_id', 'features'])
#looks good

CPU times: user 35.2 ms, sys: 5.03 ms, total: 40.2 ms
Wall time: 5.83 s


In [18]:
%%time
GBTRegD10Model = GBTRegressionModel.load(data_path + 'GBTRegD10Model_20m')
gbtr_preds = GBTRegD10Model.transform(unrated_metadata_features)

CPU times: user 4.61 ms, sys: 1.81 ms, total: 6.42 ms
Wall time: 12.8 s


In [19]:
%%time
gbtr_top_n_predictions = gbtr_preds.sort(desc('prediction')).take(num_recs)
# extract movie ids
gbtr_top_n_ids = [r[0] for r in gbtr_top_n_predictions]

gbtr_movie_recs = movies.filter(movies.item_id.isin(gbtr_top_n_ids)).select('title', 'year')
print 'GBTR Recommendations'
print gbtr_movie_recs.toPandas()

GBTR Recommendations
                                                title  year
0   Crazy Class Wakes Up, The (Hababam sinifi uyan...  1977
1                               India (Indien) (1993)  1993
2                                       Yumeji (1991)  1991
3   I Don't Want to Sleep Alone (Hei yan quan) (2006)  2006
4                               Love and Honor (2006)  2006
5                                      PlAcido (1961)  1961
6                      Stuart: A Life Backward (2007)  2007
7                            Hollow Crown, The (2012)     0
8                       Eve and the Fire Horse (2005)  2005
9   Shadows of Our Forgotten Ancestors (Tini zabut...  1964
10  Marathon Family, The (Maratonci Trce Pocasni K...  1982
11                                  Konopielka (1982)  1982
12                             Moth, The (Cma) (1980)  1980
13           Free Will, The (Freie Wille, Der) (2006)  2006
14                   It's Such a Beautiful Day (2012)  2012
CPU times: user 21.

### Full Function Recommendation Examples

In [20]:
%%time
fav_movies = ['Iron Man', 'Tinker Tailor Soldier Spy', 'Shawshank Redemption', 'Lord of the Rings (2002)', 'Harry Potter',
             'The Family Stone', 'Shaun of the Dead', 'Up', 'A View to a Kill']
new_user_input(fav_movies = fav_movies, all_ratings = ratings, 
               movies = movies, spark_context = sc, sqlContext = sqlContext,
               num_recs = 10, movies_gp = movies_gp, movies_df = movies_df)

Favorite movies in the available set
         item_id                                              title  year
item_id                                                                  
102125    102125                                  Iron Man 3 (2013)  2013
102051    102051                Iron Man: Rise Of Technovore (2013)  2013
77561      77561                                  Iron Man 2 (2010)  2010
102007    102007                    Invincible Iron Man, The (2007)  2007
59315      59315                                    Iron Man (2008)  2008
89753      89753                   Tinker Tailor Soldier Spy (2011)  2011
318          318                   Shawshank Redemption, The (1994)  1994
5952        5952      Lord of the Rings: The Two Towers, The (2002)  2002
69844      69844      Harry Potter and the Half-Blood Prince (2009)  2009
40815      40815         Harry Potter and the Goblet of Fire (2005)  2005
88125      88125  Harry Potter and the Deathly Hallows: Part 2 (...  2011
5

In [21]:
%%time
fav_movies = ['Tinker Tailor Soldier Spy', 'Shawshank Redemption', 'Lord of the Rings']
new_user_input(fav_movies = fav_movies, all_ratings = ratings, 
               movies = movies, spark_context = sc, sqlContext = sqlContext,
               num_recs = 10, movies_gp = movies_gp, movies_df = movies_df)

Favorite movies in the available set
         item_id                                              title  year
item_id                                                                  
89753      89753                   Tinker Tailor Soldier Spy (2011)  2011
318          318                   Shawshank Redemption, The (1994)  1994
2116        2116                      Lord of the Rings, The (1978)  1978
7153        7153  Lord of the Rings: The Return of the King, The...  2003
4993        4993  Lord of the Rings: The Fellowship of the Ring,...  2001
5952        5952      Lord of the Rings: The Two Towers, The (2002)  2002
Adding ratings to full set
Creating prediction set
Training ALS model
Making Predictions

ALS Recommendations
                                               title  year
0                        Unbeatable (Ji zhan) (2013)  2013
1                     Amanece, que no es poco (1989)  1989
2                National Theatre Live: Frankenstein  2014
3  I Don't Want to Sleep 

In [22]:
%%time
fav_movies = ['Frozen', 'Tangled', 'Oceans Eleven', 'Toy Story', 'The Princess Bride',  
              'The Incredibles', 'Castle in the Sky', 'Monsters, Inc']
new_user_input(fav_movies = fav_movies, all_ratings = ratings, 
               movies = movies, spark_context = sc, sqlContext = sqlContext,
               num_recs = 10, movies_gp = movies_gp, movies_df = movies_df)

Favorite movies in the available set
         item_id                                              title  year
item_id                                                                  
1              1                                   Toy Story (1995)  1995
3114        3114                                 Toy Story 2 (1999)  1999
78499      78499                                 Toy Story 3 (2010)  2010
106022    106022                         Toy Story of Terror (2013)  2013
115875    115875          Toy Story Toons: Hawaiian Vacation (2011)  2011
115879    115879                  Toy Story Toons: Small Fry (2011)  2011
120468    120468            Toy Story Toons: Partysaurus Rex (2012)  2012
120474    120474                  Toy Story That Time Forgot (2014)  2014
1197        1197                         Princess Bride, The (1987)  1987
6350        6350  Laputa: Castle in the Sky (TenkA no shiro Rapy...  1986
4886        4886                              Monsters, Inc. (2001)  2001
A

In [23]:
%%time
fav_movies = ['The Sound of Music', 'Blackhawk Down', 'Pearl Harbor', 'Toy Story', 'The Princess Bride',  
              'Foreign Student', 'Star Wars', 'The Shining', 'Rear Window', 'Groundhog Day', 'Ghostbusters', 
              'Robin Hood (1993)', 'Die Hard']
new_user_input(fav_movies = fav_movies, all_ratings = ratings, 
               movies = movies, spark_context = sc, sqlContext = sqlContext,
               num_recs = 10, movies_gp = movies_gp, movies_df = movies_df)

Favorite movies in the available set
         item_id                                              title  year
item_id                                                                  
1035        1035                         Sound of Music, The (1965)  1965
4310        4310                                Pearl Harbor (2001)  2001
1              1                                   Toy Story (1995)  1995
3114        3114                                 Toy Story 2 (1999)  1999
78499      78499                                 Toy Story 3 (2010)  2010
106022    106022                         Toy Story of Terror (2013)  2013
115875    115875          Toy Story Toons: Hawaiian Vacation (2011)  2011
115879    115879                  Toy Story Toons: Small Fry (2011)  2011
120468    120468            Toy Story Toons: Partysaurus Rex (2012)  2012
120474    120474                  Toy Story That Time Forgot (2014)  2014
1197        1197                         Princess Bride, The (1987)  1987
5